In [1]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-11-14 17:08:03.030517: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 17:08:03.039030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-14 17:08:03.048142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-14 17:08:03.050880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 17:08:03.058203: I tensorflow/core/platform/cpu_feature_guar

# Transformation de toutes les images en JPEG

In [2]:
import pathlib
import sys
sys.path.append("..")

from utilitaires.image_util import transform_image_type_to_jpg

data_dir_path = '/home/shared/Mammiferes'


data_dir = pathlib.Path(data_dir_path).with_suffix('')

image_paths = list(data_dir.glob('*/*.*'))

transform_image_type_to_jpg(image_paths)

image_count = len(image_paths)

print(image_count)


252


# Data Augmentation

In [5]:

from tensorflow.data import Dataset
import tensorflow_datasets as tfds

batch_size = 32

(train_datasets, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

image_width, image_height = (256, 256)

rng = tf.random.Generator.from_seed(123, alg='philox')


def resize_and_rescale(image, label):
    image = tf.image.resize(image, [image_width, image_height])
    image = (image / 255.0)
    image = tf.cast(image, tf.uint8)
    return image, label


def augment_and_make_seed(x, y):
    print(x)
    print(y)
    seed = rng.make_seeds(1)[:, 0]
    image, label = augment((x, y), seed)
    return image, label


def augment(image_label, seed):
    image, label = image_label
    image, label = resize_and_rescale(image, label)
    image = tf.image.resize_with_crop_or_pad(image, image_width + 6, image_height + 6)
    # Make a new seed.
    new_seed = tf.random.split(seed, num=1)[0, :]
    # Random crop back to the original size.
    image = tf.image.stateless_random_crop(
        image, size=[image_width, image_height, 3], seed=seed)
    # Random brightness.
    image = tf.image.stateless_random_brightness(
        image, max_delta=0.5, seed=new_seed)
    image = tf.clip_by_value(image, 0, 1)
    image = tf.cast(image, tf.uint8)

    return image, label


def apply_augmentation(dataset: Dataset):
    resized_dataset = dataset.map(
        resize_and_rescale
    )
    augmented_dataset = resized_dataset.map(
        augment_and_make_seed
    )
    return augmented_dataset.concatenate(dataset)


train_datasets: Dataset = train_datasets.apply(apply_augmentation).shuffle(1000).batch(batch_size)
print(train_datasets)

for batch in enumerate(train_datasets.take(5)):
    print(batch)



Tensor("args_0:0", shape=(256, 256, 3), dtype=uint8)
Tensor("args_1:0", shape=(), dtype=int64)
<_BatchDataset element_spec=(TensorSpec(shape=(None, None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
(0, (<tf.Tensor: shape=(32, 256, 256, 3), dtype=uint8, numpy=
array([[[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        ...,

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]

2024-11-14 17:11:52.524469: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
